In [1]:
%%bash 
git clone https://github.com/comp0190/tutorial_docker_big_data.git

Cloning into 'tutorial_docker_big_data'...


In [2]:
!pwd

/content


In [3]:
import os, sys

os.chdir('./tutorial_docker_big_data/mnist-cnn_v0.5')
sys.path.append('./src/')

In [4]:
!pwd

/content/tutorial_docker_big_data/mnist-cnn_v0.5


# 1. Load Config 

In [5]:
# load config file
import yaml
with open('config.yml', 'r') as file:
    config = yaml.safe_load(file)

config

{'loss': 'categorical_crossentropy',
 'optimizer': 'adam',
 'metrics': 'accuracy',
 'batch_size': 256,
 'epochs': 30,
 'validation_split': 0.2,
 'model_name': 'mnist_cnn_1.h5',
 'model_path': './models',
 'results_file': 'mnist_cnn_1_results.txt',
 'results_path': './results'}

# 2. Train Model 

In [6]:
# load dataset
from tensorflow import keras
from keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [7]:
# prepare dataset
import preprocess_dataset

processed_dataset = preprocess_dataset.process_dataset(x_train, y_train, x_test, y_test)

In [8]:
# train mocdel
import model

cnn = model.initiate_model()

In [9]:
cnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 1, 28, 8)          2016      
                                                                 
 batch_normalization (BatchN  (None, 1, 28, 8)         32        
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 1, 28, 8)          0         
                                                                 
 average_pooling2d (AverageP  (None, 1, 14, 8)         0         
 ooling2D)                                                       
                                                                 
 conv2d_1 (Conv2D)           (None, 1, 14, 16)         1152      
                                                                 
 batch_normalization_1 (Batc  (None, 1, 14, 16)        6

In [10]:
cnn.compile(loss=config['loss'], optimizer=config['optimizer'], metrics=[config['metrics']])

In [11]:
cnn.fit(
    processed_dataset['images_train'],
    processed_dataset['labels_train'],
    batch_size=config['batch_size'],
    epochs=config['epochs'],
    validation_split=config['validation_split'])

Epoch 1/30
188/188 [==============================] - 8s 31ms/step - loss: 1.2662 - accuracy: 0.6116 - val_loss: 1.7711 - val_accuracy: 0.3565
Epoch 2/30
188/188 [==============================] - 6s 34ms/step - loss: 0.4248 - accuracy: 0.8774 - val_loss: 0.4452 - val_accuracy: 0.8753
Epoch 3/30
188/188 [==============================] - 5s 28ms/step - loss: 0.2985 - accuracy: 0.9129 - val_loss: 0.2135 - val_accuracy: 0.9377
Epoch 4/30
188/188 [==============================] - 6s 34ms/step - loss: 0.2488 - accuracy: 0.9265 - val_loss: 0.1470 - val_accuracy: 0.9554
Epoch 5/30
188/188 [==============================] - 5s 27ms/step - loss: 0.2301 - accuracy: 0.9322 - val_loss: 0.1278 - val_accuracy: 0.9620
Epoch 6/30
188/188 [==============================] - 6s 34ms/step - loss: 0.2067 - accuracy: 0.9378 - val_loss: 0.1210 - val_accuracy: 0.9632
Epoch 7/30
188/188 [==============================] - 6s 34ms/step - loss: 0.1921 - accuracy: 0.9428 - val_loss: 0.1111 - val_accuracy: 0.9668

# 3. Evaluate the Model 

In [12]:
import predict

y_predict = predict.predict(cnn, processed_dataset['images_test'])

313/313 [==============================] - 1s 3ms/step


In [13]:
import numpy as np
y_true = np.argmax(processed_dataset['labels_test'], 1)
y_true

array([7, 2, 1, ..., 4, 5, 6])

In [14]:
%%bash
mkdir results

In [15]:
import metrics
metrics.evaluate_model(y_true, y_predict, output_type='print2file',
  output_path=os.path.join(config['results_path'], config['results_file']))

0

In [16]:
metrics.evaluate_model(y_true, y_predict, output_type='print')

              precision    recall  f1-score   support

           0       0.99      0.99      0.99       980
           1       0.99      0.99      0.99      1135
           2       0.97      0.98      0.98      1032
           3       0.94      0.99      0.97      1010
           4       0.99      0.96      0.98       982
           5       0.98      0.97      0.97       892
           6       0.99      0.98      0.98       958
           7       0.98      0.98      0.98      1028
           8       0.98      0.96      0.97       974
           9       0.95      0.96      0.96      1009

    accuracy                           0.98     10000
   macro avg       0.98      0.98      0.98     10000
weighted avg       0.98      0.98      0.98     10000



# 4. Save Trained Model 

In [17]:
import  os
cnn.save(os.path.join(config['model_path'], config['model_name']))

# 5. Run Python script 

In [18]:
!pwd

/content/tutorial_docker_big_data/mnist-cnn_v0.5


In [19]:
config['model_name'] = 'mnist_cnn_2.h5'
config['epochs'] = 5
config['results_file'] = 'mnist_cnn_2_results.txt'
config

{'loss': 'categorical_crossentropy',
 'optimizer': 'adam',
 'metrics': 'accuracy',
 'batch_size': 256,
 'epochs': 5,
 'validation_split': 0.2,
 'model_name': 'mnist_cnn_2.h5',
 'model_path': './models',
 'results_file': 'mnist_cnn_2_results.txt',
 'results_path': './results'}

In [20]:
%%bash 
touch 'config_experiment2.yml'

In [21]:
import yaml
with open('config_experiment2.yml', 'w') as file:
  print(yaml.safe_dump(config, sort_keys=False), file=file)

Change in train_model.py

`# load config file`

`import yaml`

`with open('config_experiment2.yml', 'r') as file:`

`     config = yaml.safe_load(file)`

In [22]:
!python3 train_model.py

2023-03-13 03:16:45.162061: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-13 03:16:45.162218: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-13 03:16:45.162246: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
2023-03-13 03:16:47.731466: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to c

In [23]:
!python3 run.py

2023-03-13 03:17:35.862472: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-13 03:17:35.862615: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-13 03:17:35.862643: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
2023-03-13 03:17:38.332583: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to c

## Exercise: Predict Labels 
1. Randomly sample 50 images from test dataset
2. Evaluate the sample using the trained model
3. Store the results into file 'mnist-cnn/results/results_sample_50.txt'
4. Evaluate the sample using both trained models 

# 6. Config File as an Argumemt 

In [ ]:
os.chdir('./tutorial_docker_big_data/mnist-cnn/')
sys.path.append('./src/')

In [81]:
!python3 run.py --config 'config.yml'

2023-03-13 04:40:55.378916: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-13 04:40:55.379035: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-13 04:40:55.379058: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
{'loss': 'categorical_crossentropy', 'optimizer': 'adam', 'metrics': 'accuracy', 'batch_size': 256, 'epochs': 3

In [ ]:
!python3 run.py --config '../mnist-cnn_v0.5/config_experiment2.yml'